In [1]:
import pandas as pd
import torch
from torch import nn, Tensor
import math
import time
import copy
import io
import numpy as np
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import tensorflow as tf
import datetime
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
def load_pre_trained_embeddings(path: str):
    fin = io.open(path, 'r', encoding='utf-8', newline='\n', errors='ignore')
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = map(float, tokens[1:])
    return data

In [3]:
def load_words(file_name):
    df = pd.read_csv(file_name)
    text = df[['question1','question2']]
    targets = df['is_duplicate']
    return text, targets

In [4]:
def get_embedding(vocab):
    matrix_len = len(vocab)
    emb_dim = 300
    weights_matrix = np.zeros((matrix_len, emb_dim))
    words_found = 0
    for i, word in enumerate(vocab):
        try: 
            weights_matrix[i] = np.array(list(vocab[word]))
            words_found += 1
        except KeyError:
            weights_matrix[i] = np.random.normal(scale=0.6, size=emb_dim)
        except ValueError:
            weights_matrix[i] = np.random.normal(scale=0.6, size=emb_dim)
    return torch.from_numpy(weights_matrix)

In [5]:
def create_emb_layer(weights_matrix, non_trainable=False):
    num_embeddings, embedding_dim = weights_matrix.size()
    emb_layer = nn.Embedding(num_embeddings, embedding_dim)
    emb_layer.load_state_dict({'weight': weights_matrix})
    if non_trainable:
        emb_layer.weight.requires_grad = False
    return emb_layer, num_embeddings, embedding_dim

In [6]:
def pad_to_300x300(tensor):
    width = tensor.shape[1]
    pad_width = max(0, 300 - width)
    pad_left = pad_width // 2
    pad_right = pad_width - pad_left
    padded_tensor = F.pad(tensor, (pad_left, pad_right, 0, 0), mode='constant', value=0)
    return padded_tensor

In [7]:
tokens_dict = load_pre_trained_embeddings('/Users/stavbracha/PycharmProjects/crawl-300d-2M-subword/crawl-300d-2M-subword.vec')

In [8]:
class Dataset(Dataset):
    def __init__( self, file_name):
        self.words, self.targets = load_words(file_name)
        self.uniq_words = self.get_uniq_words()
        self.token_dict = tokens_dict
        self.tokenized_words = self.word2token()
        
    def get_uniq_words(self):
        result = set()
        self.words.dropna(subset=['question1'], inplace=True)
        self.words['question1'].str.lower().str.split().apply(result.update)
        self.words.dropna(subset=['question2'], inplace=True)
        self.words['question2'].str.lower().str.split().apply(result.update)
        return list(result)
    
    def word2token(self):
        self.words['question1_list'] = self.words['question1'].str.lower().str.split()
        self.words['question2_list'] = self.words['question2'].str.lower().str.split()
        word_to_index = {word: index for index, word in enumerate(self.token_dict)}
        df_models1 = pd.DataFrame(self.words['question1_list'].tolist()).fillna('').add_prefix('model_')
        df_models1 = df_models1.apply(lambda x: x.map(word_to_index.get)).apply(pd.to_numeric,errors='coerce')
        df_models2 = pd.DataFrame(self.words['question2_list'].tolist()).fillna('').add_prefix('model_')
        df_models2 = df_models2.apply(lambda x: x.map(word_to_index.get)).apply(pd.to_numeric,errors='coerce')
        return torch.tensor(df_models1.values), torch.tensor(df_models2.values)
        
    def __len__(self):
        return len(self.words)

    def __getitem__(self, index):
        return (
            self.tokenized_words[0][index,:], self.tokenized_words[1][index,:],
            torch.tensor(self.targets[index]),
        )

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
train_DataSet = Dataset('train_data.csv')
valid_DataSet = Dataset('valid_data.csv')
test_DataSet = Dataset('test_data.csv')

In [11]:
class MultiHeadCoAttention(nn.Module):

    def __init__(self, d_model, n_head):
        super(MultiHeadCoAttention, self).__init__()
        self.n_head = n_head
        self.softmax = nn.Softmax(dim=-1)
        self.quest_1 = nn.Linear(d_model, d_model)
        self.quest_2 = nn.Linear(d_model, d_model)

    def split(self, tensor):
        batch_size, length, d_model = tensor.size()
        d_tensor = d_model // self.n_head
        tensor = tensor.view(batch_size, length, self.n_head, d_tensor).transpose(1, 2)
        return tensor

    def concat(self, tensor):
        batch_size, head, length, d_tensor = tensor.size()
        d_model = head * d_tensor
        tensor = tensor.transpose(1, 2).contiguous().view(batch_size, length, d_model)
        return tensor

    def forward(self, q1, k1, v1, q2, k2, v2):
        
        q1, k1, v1 = self.quest_1(q1), self.quest_1(k1), self.quest_1(v1)
        q1, k1, v1 = self.split(q1), self.split(k1), self.split(v1)
        q2, k2, v2 = self.quest_2(q2), self.quest_2(k2), self.quest_2(v2)
        q2, k2, v2 = self.split(q2), self.split(k2), self.split(v2)
        head, n_split, length, d_tensor= k1.size() 
        k1_t = k1.transpose(2,3)
        k2_t = k2.transpose(2,3)
        score1 = torch.matmul(q1,k1_t) / math.sqrt(d_tensor)
        score2 = torch.matmul(q2,k2_t) / math.sqrt(d_tensor)
        score1 = self.softmax(score1)
        score2 = self.softmax(score2)
        v1 = torch.matmul(score1, v2)
        v2 = torch.matmul(score2, v1)
        out1 = self.concat(v1)
        out2 = self.concat(v2)
        return out1, out2

In [12]:
class EncoderLayer(nn.Module):

    def __init__(self, d_model, ffn_hidden, n_head, drop_prob):
        super(EncoderLayer, self).__init__()
        self.attention = MultiHeadCoAttention(d_model=d_model, n_head=n_head)
        self.norm1 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(p=drop_prob)
        self.ffn = PositionwiseFeedForward(d_model=d_model, hidden=ffn_hidden, drop_prob=drop_prob)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout2 = nn.Dropout(p=drop_prob)

    def forward(self, x1, x2):
        _x1 = x1
        _x2 = x2
        x1, x2 = self.attention(q1=x1, k1=x1, v1=x1, q2=x2, k2=x2, v2=x2)
        
        x1 = self.dropout1(x1)
        x2 = self.dropout1(x2)
        x1 = self.norm1(x1 + _x1)
        x2 = self.norm1(x2 + _x2)
        
        _x1 = x1
        _x2 = x2
        x1 = self.ffn(x1)
        x2 = self.ffn(x2)
      
        x1 = self.dropout2(x1)
        x2 = self.dropout2(x2)
        x1 = self.norm2(x1 + _x1)
        x2 = self.norm2(x2 + _x2)
        return x1, x2

In [13]:
class Encoder(nn.Module):

    def __init__(self, d_model, ffn_hidden, n_head, n_layers, drop_prob, weights_matrix):
        super().__init__()
        
        self.emb, num_embeddings, embedding_dim = create_emb_layer(weights_matrix)
        self.layers = nn.ModuleList([EncoderLayer(d_model=d_model,
                                                  ffn_hidden=ffn_hidden,
                                                  n_head=n_head,
                                                  drop_prob=drop_prob)
                                     for _ in range(n_layers)])
        self.linear2output = nn.Linear(d_model, 1)

    def forward(self, x1, x2):
        x1 = self.emb(x1.long())
        x2 = self.emb(x2.long())
        for layer in self.layers:
            x1, x2 = layer(x1, x2)
        
        x1 = self.linear2output(x1)
        x2 = self.linear2output(x2)
        return x1, x2

In [14]:
class PositionwiseFeedForward(nn.Module):

    def __init__(self, d_model, hidden, drop_prob=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.linear1 = nn.Linear(d_model, hidden)
        self.linear2 = nn.Linear(hidden, d_model)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=drop_prob)

    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear2(x)
        return x

In [15]:
class ContrastiveLoss(torch.nn.Module):
    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2, keepdim = True)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      label * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))
        
        return loss_contrastive

In [16]:
dim_model = 300
d_hid = 300 
nlayers = 2 
nhead = 3 
dropout = 0.2 
weights = get_embedding(train_DataSet.token_dict)
model = Encoder(d_model=dim_model, n_head=nhead, ffn_hidden=d_hid, n_layers=nlayers, drop_prob=dropout, weights_matrix=weights).to(device)

In [24]:
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_CosineSimilarity = tf.keras.metrics.CosineSimilarity('train_CosineSimilarity')
valid_loss = tf.keras.metrics.Mean('valid_loss', dtype=tf.float32)
valid_CosineSimilarity = tf.keras.metrics.CosineSimilarity('valid_CosineSimilarity')

In [28]:
criterion = ContrastiveLoss()
lr = 0.5 
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.95)

def train(model: nn.Module, training_data):
    epoch_loss = 0
    model.train()
    total_loss = 0

    for i, (input1, input2, label) in enumerate(training_data):
        optimizer.zero_grad()
        output1, output2 = model(pad_to_300x300(input1),pad_to_300x300(input2))
        loss = criterion(output1,output2,label)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_loss(loss.item())
        train_CosineSimilarity(output1.detach().numpy(),output2.detach().numpy())
    return epoch_loss


def evaluate(model: nn.Module, eval_data: Tensor):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for i, (input1, input2, label) in enumerate(eval_data):
            output1, output2 = model(pad_to_300x300(input1),pad_to_300x300(input2))
            loss = criterion(output1,output2,label)    
            total_loss += loss.item()
            valid_loss(loss.item())
            valid_CosineSimilarity(output1,output2)
    return total_loss/(len(eval_data)-1), cosine_similarity(output1.detach().numpy(),output2.detach().numpy())

In [29]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
valid_log_dir = 'logs/gradient_tape/' + current_time + '/valid'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
valid_summary_writer = tf.summary.create_file_writer(valid_log_dir)

In [30]:
batch_size = 30
eval_batch_size = 15
best_val_loss = float('inf')
epochs = 7
best_model = None
train_epoch_loss = []
val_epoch_loss = []

for epoch in range(1, epochs + 1):
    train_data = DataLoader(train_DataSet, batch_size = batch_size)
    valid_data = DataLoader(valid_DataSet, batch_size = eval_batch_size)
    epoch_start_time = time.time()
    train_epoch_loss.append(train(model, train_data))
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', train_loss.result(), step=epoch)
        tf.summary.scalar('CosineSimilarity', train_CosineSimilarity().result(), step=epoch)
    tmp_loss, valid_CosineSimilarity = (evaluate(model, valid_data))
    val_epoch_loss.append(tmp_loss)
    with valid_summary_writer.as_default():
        tf.summary.scalar('loss', valid_loss.result(), step=epoch)
        tf.summary.scalar('CosineSimilarity', valid_CosineSimilarity().result(), step=epoch)
    elapsed = time.time() - epoch_start_time
    print('-' * 89)
    print(f'| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | '
          f'valid loss {val_epoch_loss[-1]:5.2f} | valid CosineSimilarity {valid_CosineSimilarity:8.2f}')
    print('-' * 89)

    if val_epoch_loss[-1] < best_val_loss:
        best_val_loss = val_epoch_loss[-1]
        best_model = copy.deepcopy(model)
        
    train_loss.reset_states()
    valid_loss.reset_states()
    train_CosineSimilarity.reset_states()
    valid_CosineSimilarity.reset_states()

save_path = 'trained_NLP_model.pt'
torch.save(model.state_dict(), save_path)

KeyboardInterrupt: 

In [ ]:
def test(model: nn.Module, testing_data: Tensor):
    model.eval()
    test_scores = []
    with torch.no_grad():
        for i, (input1, input2, label) in enumerate(testing_data):
            output1, output2 = model(pad_to_300x300(input1),pad_to_300x300(input2))
            test_scores.append(cosine_similarity(output1.detach().numpy(),output2.detach().numpy()))
    test_total_cosinesimilarity = sum(test_scores) / len(test_scores)
    print('-' * 89)
    print(f'test CosineSimilarity {test_total_cosinesimilarity:8.2f}')
    print('-' * 89)

In [ ]:
test_data = DataLoader(test_DataSet.tokenized_words, batch_size = batch_size)
test(model,test_data)

## Text Answer

The model includes 2 encoding layers with co-attention and a pre-trained embedding layer that was trained using fastText. The purpose of using co-attention is to capture the influence of each sentence on the other. Apart from attention, both sentences pass through the exact same weights in each iteration. The loss function that I used is ContrastiveLoss, which computes the Euclidean distance for each pair of sentence embeddings. The criterion for measuring the performance of the model was cosine similarity.
